# Experiments: Classical ML

In [9]:
# Import all dependencies
import pandas as pd

# Load data
data= pd.read_csv("../data/tier2_data.csv")
data = data.drop("Unnamed: 0",axis=1)
data.head(10)

,url,label,url_length,num_digits,num_special_chars,dot_count,hyphen_in_domain,at_symbol,double_slash,subdomain_count,domain_length,path_length,ip_flag,keyword_flag,url_entropy,domain_length_t2,malicious_tld_flag,encoded_flag,path_length_t2
0,https://hlaly.rouseauhealth.com/qiwlr,1,37.0,0.0,6.0,2.0,0.0,0.0,0.0,1,23.0,6.0,0.0,0.0,4.067165,23,0,0,6
1,https://allegro.pl-oferta814932.sbs,1,35.0,6.0,6.0,2.0,1.0,0.0,0.0,1,27.0,0.0,0.0,0.0,4.321721,27,0,0,0
2,https://hinnf.rouseauhealth.com/qiwlr,1,37.0,0.0,6.0,2.0,0.0,0.0,0.0,1,23.0,6.0,0.0,0.0,4.195676,23,0,0,6
3,http://allegrolokalnie.oferty-kategoria-propon...,1,73.0,5.0,10.0,2.0,1.0,0.0,0.0,1,66.0,0.0,0.0,0.0,4.394598,66,0,0,0
4,http://allegrolokalnie.oferty-kategoria-propon...,1,73.0,5.0,10.0,2.0,1.0,0.0,0.0,1,66.0,0.0,0.0,0.0,4.356859,66,0,0,0
5,http://allegrolokalne.pl73467064173.cfd,1,39.0,11.0,5.0,2.0,0.0,0.0,0.0,1,32.0,0.0,0.0,0.0,4.352978,32,0,0,0
6,https://allegrolokalnie.oferta-23942-dla-ciebi...,1,63.0,5.0,9.0,2.0,1.0,0.0,0.0,1,55.0,0.0,0.0,0.0,4.433496,55,0,0,0
7,https://cgebsj.cfd/indexco.jp,1,29.0,0.0,6.0,2.0,0.0,0.0,0.0,0,10.0,11.0,0.0,0.0,4.047299,10,0,0,11
8,https://cghebsl.cfd/login,1,25.0,0.0,5.0,1.0,0.0,0.0,0.0,0,11.0,6.0,0.0,1.0,3.973661,11,0,0,6
9,https://foliox.cfd/login,1,24.0,0.0,5.0,1.0,0.0,0.0,0.0,0,10.0,6.0,0.0,1.0,3.855389,10,0,0,6


## Models with engineered features